In [1]:
import numpy as np, scipy.stats as st

import src
import src.tours as tours
from src.cross_entropy import CrossEntropy


In [10]:
# drone tour parameters
num_sites, num_drones = 10, 4
sites = np.random.rand(num_sites, 2)  # the 2 indicates we are sampling sites from the plane

drone_tour = tours.DroneTour(sites, num_drones)

# cross entropy parameters
x_entropy_sample_size = 5 # sample size
x_entropy_quantile = 0.2  # specifies the 90% percentile

x_entropy_estimator = CrossEntropy(drone_tour, x_entropy_sample_size, x_entropy_quantile)

# an example way to use the callback -- see below for details
sample_scores = []
hundreth_tour_costs = []

In [7]:
def callback(**kwargs):
    """

    :param kwargs:
    The keyword arguments passed to callback are
            - iteration, the current iteration
            - sample_states, the current sample of states in the given round of annealing
            - sample_scores, the scores associated with the sampled states in the given round of annealing
            - threshold, the scores which form the (1-quantile) quantile of the sample scores
            - distribution_parameters, the current parameters of the distribution of states in the state space
    :return:
    """
    sample_scores.append(kwargs['sample_scores'])  # for example, maybe we want a list of all sample scores during
    # the run of cross entropy

In [11]:
for i in range(0,10):
    sample_scores = []
    x_entropy_estimator.minimize(callback=callback)
    hundreth_tour_costs.append(min(sample_scores[99]))
    
print(hundreth_tour_costs)
#print(len(sample_scores))

[2.5113449455241312, 1.9311228398076326, 1.7465253485980547, 2.0167493386745075, 1.7455019302313026, 2.024112894338163, 2.4640445230780474, 2.0631805868797817, 2.4896254592039404, 2.1380652563101186]


In [5]:
st.t.interval(0.95, len(hundreth_tour_costs)-1, loc=np.mean(hundreth_tour_costs), scale=st.sem(hundreth_tour_costs))

(2.5559624399056973, 3.152910121628667)